# **PROBLEMA DE NEGOCIO**


---




La necesidad de prever y optimizar el gasto de sus usuarios ha llevado a una empresa de comercio electrónico a buscar soluciones innovadoras. Como científicos de datos, hemos sido convocados para desarrollar un modelo de machine learning que pueda predecir con precisión cuánto gastará un usuario al visitar dicho sitio web.

### **Tus tareas principales serán:**

**1. Preprocesamiento de Datos:** Importar correctamente y analizar y comprender el conjunto de datos proporcionado, realizar limpieza de datos, eliminar atributos que no aportan valor y manejar valores faltantes.

**2. Exploración y Feature Engineering:** Realizar visualizaciones para entender las relaciones entre las variables y seleccionar las características relevantes, identificar variables llaves, codificación de variables categóricas y normalización/escalado de datos.

**3. Construcción de Modelos:** Experimentar con algunos algoritmos de machine learning como Linear Regression, Decision Tree Regressor, Random Forest Regressor, entre otros.

**4. Evaluación y Selección del Modelo:** Evaluar los modelos utilizando métricas como el error cuadrático medio (MSE), la raíz cuadrada del error cuadrático medio (RMSE) y el coeficiente de determinación (R²). Seleccionar el modelo con el mejor rendimiento para la predicción del gasto de los usuarios.

## **1. Configuración del Ambiente**


---




In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import r2_score, mean_squared_error, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import xgboost as xgb
import lightgbm as lgb
# Si se usa colab hay que instalar catboost
try:
    import catboost as cat
except Exception:
    !pip install catboost
finally:
    import catboost as cat

from datetime import date
from joblib import dump, load

import warnings; warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

## **2. Preprocesamiento de Datos**


---


In [ ]:
# # Solo la primera vez, para descargar el archivo
df = pd.read_csv('https://raw.githubusercontent.com/ElProfeAlejo/Bootcamp_Databases/main/traffic_site.csv')
df.to_csv('traffic_site.csv', index=False)

In [ ]:
# # Cargamos el archivo ya descargado, cambiando el tipo de dato de algunas las columnas
# df = pd.read_csv('traffic_site.csv', parse_dates=['date'], dtype={'fullVisitorId': 'object', 'visitId': 'object'})

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Convertimos los diccionarios en nuevas columnas
diccionarios = ['device','geoNetwork','trafficSource','totals']

for columna in diccionarios: 
    df = df.join(pd.DataFrame([json.loads(linea) for linea in df[columna]]))

# Eliminamos las originales
df.drop(columns=diccionarios, inplace=True)

In [ ]:
# Convertimos la columna 'adwordsClickInfo' que se acaba de crear en nuevas columnas
df = df.join(pd.DataFrame([json.loads(json.dumps(linea)) for linea in df['adwordsClickInfo']]))

# Eliminamos las originales
df.drop(columns=['adwordsClickInfo'], inplace=True)

In [ ]:
df.info()

In [ ]:
# Nulos en nuestra variable objetivo
df.transactionRevenue.isna().sum()

In [ ]:
# Filtramos los datos que no son nan, observemos que no hay ceros por lo que inferimos que los nulos son los ceros
df[~df.transactionRevenue.isna()].transactionRevenue.unique()

In [ ]:
# Rellenamos los nulos con 0 para la variable objetivo y dividimos por 1000000 para que quede en millones
df.transactionRevenue.fillna(0, inplace=True)
df['transactionRevenue'] = pd.to_numeric(df['transactionRevenue']) / 1000000

In [ ]:
# Eliminarla la columna en caso de tener un solo valor
eliminadas = 0
columnas = []
for columna in df.columns:
    if len(df[columna].value_counts()) == 1:
        df.drop(columns=columna, inplace=True)
        eliminadas += 1
        columnas.append(columna)
print(f'Se eliminaron {eliminadas} columnas')
print(f'Las columnas eliminadas fueron: {columnas}')

In [ ]:
# Contamos valores diferentes en cada columna
df.nunique(dropna=False)

In [ ]:
# Analizamos las columnas que tengan nan
df.isna().sum().sort_values(ascending=False)

In [ ]:
# Contamos sus valores
df['referralPath'].value_counts()

In [ ]:
df['keyword'].value_counts()

In [ ]:
df['gclId'].value_counts()

In [ ]:
df['page'].value_counts()

In [ ]:
df['slot'].value_counts()

In [ ]:
df['adContent'].value_counts()

In [ ]:
# Se decide eliminar estas columnas con nulos porque no aportan información
df.drop(columns=['referralPath', 'keyword', 'adContent', 'slot', 'page', 'gclId'], inplace=True)

In [ ]:
# Analizamos valores duplicados
print(df.duplicated().sum())

# Columnas duplicadas en caso de tenerlas
if df.duplicated().sum() > 0:
    print('Se encontraron duplicados')
    print(df[df.duplicated()])
else:
    print('No se encontraron duplicados')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
# Modificamos el tipo de datos de algunas columnas
df['visitStartTime'] = pd.to_datetime(df['visitStartTime'], unit='s') # tiempo en segundos
df['hits'] = df['hits'].astype(int)
df['pageviews'] = df['pageviews'].astype(int)
df['transactionRevenue'] = df['transactionRevenue'].astype(float)

## **3. Exploración y Feature Engineering**


---


In [ ]:
# Grafico de dispersión (antes)
plt.figure(figsize=(10, 6))
plt.scatter(range(len(df['transactionRevenue'])), df['transactionRevenue'], alpha=0.5)
plt.title('Gráfico de Dispersión de transactionRevenue')
plt.xlabel('Índice')
plt.ylabel('transactionRevenue')
plt.show()

In [ ]:
sns.boxplot(df.transactionRevenue)

In [ ]:
df.describe()

In [ ]:
df['transactionRevenue'].value_counts().sort_values(ascending=False)

In [ ]:
#Histograma (antes)
ax = sns.histplot(data=df, x='transactionRevenue', kde=False)
ax.set_title('Histograma de transactionRevenue')
ax.set_xlabel('transactionRevenue')

In [ ]:
def feature_engineering(df):
    
    df['visitStartTime'] = pd.to_datetime(df['visitStartTime'],unit="s")
    df['visitStartTime_year'] = df['visitStartTime'].apply(lambda x: x.year)
    df['visitStartTime_month'] = df['visitStartTime'].apply(lambda x: x.month)
    df['visitStartTime_day'] = df['visitStartTime'].apply(lambda x: x.day)
    df['visitStartTime_weekday'] = df['visitStartTime'].dt.day_name()
    # df['visitStartTime_hour'] = df['visitStartTime'].apply(lambda x : x.hour)
    df['visitStartTime_range'] = pd.cut(df['visitStartTime'].dt.hour, 
                                    bins=[0, 6, 12, 18, 24], 
                                    labels=['madrugada', 'mañana', 'tarde', 'noche'], 
                                    ordered=False).astype('object')

    df.drop(columns=['date', 'fullVisitorId', 'sessionId', 'visitId', 'visitStartTime'], inplace=True)

In [ ]:
feature_engineering(df)
df.sample(5)

In [ ]:
# datos de source
df['source'].value_counts()

In [ ]:
# datos de deviceCategory
df['deviceCategory'].value_counts()

In [ ]:
# datos de medium
df['medium'].value_counts()

In [ ]:
# datos de channelGrouping
df['channelGrouping'].value_counts()

In [ ]:
# Tabla cruzada de country y transactionRevenue cuando es diferente de cero
pd.crosstab(df['country'], df['transactionRevenue'] != 0).sort_values(by=True, ascending=False)

In [ ]:
df.info()

In [ ]:
# Borramos mas variables
df.drop(columns=['country', 'continent', 'subContinent', 'region',	'metro', 'city', 'networkDomain', 'campaign', 'medium', 'isMobile', 'browser', 'source'], inplace=True)

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
plt.figure(figsize=(30, 10))
heatmap = sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt=".2f", vmin=-1)
heatmap.tick_params(axis='both', which='major', labelsize=14)
plt.title('Mapa de Calor de Correlaciones', fontsize=18)
plt.show()

In [ ]:
# Borramos mas variables
df.drop(columns=['pageviews', 'visitStartTime_year'], inplace=True)

In [ ]:
plt.figure(figsize=(30, 10))
heatmap = sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt=".2f", vmin=-1)
heatmap.tick_params(axis='both', which='major', labelsize=14)
plt.title('Mapa de Calor de Correlaciones', fontsize=18)
plt.show()

---

In [ ]:
# Dummizar variables categóricas
df = pd.get_dummies(df, drop_first=True)

In [ ]:
plt.figure(figsize=(30, 10))
heatmap = sns.heatmap(df.corr(numeric_only=True), annot=True, cmap='coolwarm', fmt=".2f", vmin=-1)
heatmap.tick_params(axis='both', which='major', labelsize=14)
plt.title('Mapa de Calor de Correlaciones', fontsize=18)
plt.show()

In [ ]:
# Correlaciones con la variable objetivo ordenadas de mayor a menor en valor absoluto
df.corr()['transactionRevenue'].abs().sort_values(ascending=False)

In [ ]:
# Correlacion entre las variables ordenadas de mayor a menor
correlacion = df.corr().abs().unstack().sort_values(ascending=False).drop_duplicates()
correlacion[correlacion != 1]

In [ ]:
df.shape

In [ ]:
df.to_csv('traffic_site_YO.csv', index=False)

---